In [1]:
import os
import pandas as pd
from pprint import pprint
from scipy.stats import wilcoxon
from itertools import combinations, product

In [2]:
RESULT_PATH = "./wilcoxon/"
os.makedirs(RESULT_PATH, exist_ok=True)

In [3]:
def get_data(path="./reports/"):
    df = pd.concat([
        pd.read_csv(path+file, index_col=0)
        for file in os.listdir(path)
    ])
    df = df[df.task == "gap_filling"]
    df = df.drop("dataset", axis=1)
    df = df[df.run_type != "biological validation"]
    return df

In [4]:
df = get_data()

In [5]:
def compare_2_models(d1, d2, p_treshold = 0.05):
    x = d1.accuracy_score.values
    y = d2.accuracy_score.values
    diff = x - y
    if diff.sum() == 0:
        return {
        "win":0,
        "tie":1,
        "lose":0,
        "pvalue":0,
        }
    stats, pvalue = wilcoxon(x, y)
    if pvalue <= p_treshold:
        if (diff > 0).mean() > 0.5:
            win = 1
            lose = 0
        else:
            win = 0
            lose = 1
        tie = 0
    else:
        tie = 1
        win = 0
        lose = 0
    return {
        "win":win,
        "tie":tie,
        "lose":lose,
        "pvalue":pvalue,
    }

In [6]:
combined_group = {
    key:val
    for key, val in df.groupby(["model", "trained_on"])
}

In [7]:
res = pd.DataFrame([
    {
        "model1":m1,
        "model2":m2,
        "train1":t1,
        "train2":t2,
        **compare_2_models(combined_group[(m1, t1)], combined_group[(m2, t2)]),
    }
    for (m1, m2), (t1, t2) in product(combinations(set(df.model), 2), product(set(df.trained_on), set(df.trained_on)))
])
res.to_csv(RESULT_PATH+"combined.csv")
res

,model1,model2,train1,train2,win,tie,lose,pvalue
0,cnn_1000,cnn_200,single_gap,single_gap,0,1,0,0.247145
1,cnn_1000,cnn_200,single_gap,multivariate_gaps,0,0,1,0.003182
2,cnn_1000,cnn_200,multivariate_gaps,single_gap,1,0,0,0.047858
3,cnn_1000,cnn_200,multivariate_gaps,multivariate_gaps,0,0,1,0.012374
4,cnn_1000,cae_500,single_gap,single_gap,1,0,0,0.043804
5,cnn_1000,cae_500,single_gap,multivariate_gaps,1,0,0,0.043804
6,cnn_1000,cae_500,multivariate_gaps,single_gap,1,0,0,0.000089
7,cnn_1000,cae_500,multivariate_gaps,multivariate_gaps,1,0,0,0.000089
8,cnn_1000,cae_200,single_gap,single_gap,1,0,0,0.025094
9,cnn_1000,cae_200,single_gap,multivariate_gaps,1,0,0,0.025094


In [8]:
res = pd.DataFrame([
    {
        "model1":m1,
        "model2":m2,
        "train":t,
        **compare_2_models(combined_group[(m1, t)], combined_group[(m2, t)]),
    }
    for (m1, m2), t in product(combinations(set(df.model), 2), set(df.trained_on))
])
res.to_csv(RESULT_PATH+"model.csv")
res

,model1,model2,train,win,tie,lose,pvalue
0,cnn_1000,cnn_200,single_gap,0,1,0,0.247145
1,cnn_1000,cnn_200,multivariate_gaps,0,0,1,0.012374
2,cnn_1000,cae_500,single_gap,1,0,0,0.043804
3,cnn_1000,cae_500,multivariate_gaps,1,0,0,0.000089
4,cnn_1000,cae_200,single_gap,1,0,0,0.025094
5,cnn_1000,cae_200,multivariate_gaps,1,0,0,0.000089
6,cnn_1000,cnn_500,single_gap,0,1,0,0.601213
7,cnn_1000,cnn_500,multivariate_gaps,0,0,1,0.047858
8,cnn_1000,cae_1000,single_gap,0,1,0,0.881293
9,cnn_1000,cae_1000,multivariate_gaps,1,0,0,0.000089


In [9]:
res = pd.DataFrame([
    {
        "model":m,
        "train1":t1,
        "train2":t2,
        **compare_2_models(combined_group[(m, t1)], combined_group[(m, t2)]),
    }
    for m, (t1, t2) in product(set(df.model), combinations(set(df.trained_on), 2))
])
res.to_csv(RESULT_PATH+"training.csv")
res

,model,train1,train2,win,tie,lose,pvalue
0,cnn_1000,single_gap,multivariate_gaps,0,1,0,0.056915
1,cnn_200,single_gap,multivariate_gaps,0,0,1,0.022769
2,cae_500,single_gap,multivariate_gaps,0,1,0,0.295878
3,cae_200,single_gap,multivariate_gaps,0,0,1,0.013741
4,cnn_500,single_gap,multivariate_gaps,0,0,1,0.020633
5,cae_1000,single_gap,multivariate_gaps,0,1,0,0.550292
